In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
sql1 = textwrap.dedent("""
    IF OBJECT_ID('tempdb..#Intensity') IS NOT NULL
        DROP TABLE #Intensity
""")
sql2 = textwrap.dedent("""
    SELECT userid
          ,LEFT(date, 8) AS [date]
          ,COUNT(url) AS Intensity
    INTO #Intensity
    FROM [data].[dbo].[health_records]
    GROUP BY userid, LEFT(date, 8)
    ORDER BY userid, LEFT(date, 8)
""")
sql3 = textwrap.dedent("""
    SELECT Intensity
          ,COUNT(Intensity)
    FROM #Intensity
    GROUP BY Intensity
    ORDER BY Intensity
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
cursor.execute(sql1)
cursor.execute(sql2)
rows=cursor.execute(sql3).fetchall()

In [ ]:
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
df.rename(columns={0:'Intensity', 1: 'Sessions'}, inplace=True)

df

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_key')

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
trace = go.Scatter(
    x = df['Intensity'],
    y = df['Sessions'],
    mode = 'lines',
)

data = [trace]

layout = go.Layout(
    xaxis=dict(
        title='Intensity of Use',
        titlefont=dict(
            size=16,
        ),
        ticklen=2,
        tickfont=dict(
            size=12,
        ),
        nticks=8,
        range=[0,100],
    ),
    yaxis=dict(
        title='Number of Sessions',
        titlefont=dict(
            size=16,
        ),
        tickfont=dict(
            size=11,
        ),
        nticks=20,
    ),
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='Number of Visits each Session'), image='png')

In [ ]:
py.iplot(fig, filename='Number of Visits each Session')